# DLOps Lab Assignment - 8

    Ayush Abrol B20AI052

---

### Necessary Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils, datasets, models
from torch.utils.data.sampler import SubsetRandomSampler
import warnings
warnings.filterwarnings("ignore")

import random
import wandb
import math

/home/ayushabrol/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


### Setting the device to GPU

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Logging in to wandb using my API key

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ayushabrol13. Use `wandb login --relogin` to force relogin


True

### Setting up the hyperparameters for tuning for different activation functions and optimizers

In [4]:
hyperparams = {
    "num_epochs": 30,
    "batch_size": 128,
    "learning_rate": 3e-4,
    "activation": ["Relu", "Leaky_relu", "Sigmoid", "Tanh"],
    "optimizer": ["Adam", "SGD", "RMSprop", "Adagrad"],
}

### Importing the CIFA10 dataset and selecting the even classes

Also creating train and test dataloaders

In [5]:
transform = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(size=32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)

even_indices = []
for i in range(len(dataset.targets)):
    if dataset.targets[i] % 2 == 0:
        even_indices.append(i)

train_sampler = SubsetRandomSampler(even_indices)

train_loader = DataLoader(
    dataset,
    batch_size=hyperparams["batch_size"],
    sampler=train_sampler,
    num_workers=2,
    pin_memory=True,
)

test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

test_even_indices = []
for i in range(len(test_dataset.targets)):
    if test_dataset.targets[i] % 2 == 0:
        test_even_indices.append(i)

test_sampler = SubsetRandomSampler(test_even_indices)

test_loader = DataLoader(
    test_dataset,
    batch_size=hyperparams["batch_size"],
    sampler=test_sampler,
    num_workers=2,
    pin_memory=True,
)

Files already downloaded and verified
Files already downloaded and verified


### Function for logging predicted images along with the ground truth labels to wandb

In [6]:
# This function defines the log image model
def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

### Importing torchvision's pretrained model, setting the activation function as a parameter and setting the fc layer to output 5 classes

In [7]:
class ResNet18(nn.Module):
    def __init__(self, activation):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 10)
        self.activation = activation
        self.model = self.model.to(device)

    def forward(self, x):
        if self.activation == "Relu":
            x = F.relu(self.model(x))
        elif self.activation == "Leaky_relu":
            x = F.leaky_relu(self.model(x))
        elif self.activation == "Sigmoid":
            x = F.sigmoid(self.model(x))
        elif self.activation == "Tanh":
            x = F.tanh(self.model(x))
        return x

### Function for testing the model

In [8]:
def evaluate(model, test_loader, criterion, test_loss, test_acc, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    running_loss = 0.0
    with torch.inference_mode():
        correct = 0
        for i, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            running_loss += criterion(output, target)
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == target).sum().item()
            if log_images and i == batch_idx:
                log_image_table(data, predicted, target, F.softmax(output, dim=1))

        test_loss.append(running_loss / len(test_loader))
        test_acc.append(correct / len(test_loader.dataset))
    return running_loss / len(test_loader), correct / len(test_loader.dataset)

### Function for training the model and logging the metrics to wandb

Also, initializing the wandb project and logging the hyperparameters

In [10]:
for _ in range(6):
    activation_function = random.choice(hyperparams["activation"])
    optimizer_ = random.choice(hyperparams["optimizer"])
    print(f"Activation: {activation_function}, Optimizer: {optimizer_}")
    wandb.init(
        project="B20AI052_Lab_Assignment_8",
        config={
            "num_epochs": hyperparams["num_epochs"],
            "batch_size": hyperparams["batch_size"],
            "learning_rate": hyperparams["learning_rate"],
            "activation": activation_function,
            "optimizer": optimizer_,
        },
        name=f"ResNet18_{activation_function}_{optimizer_}",
    )

    config = wandb.config

    n_steps_per_epoch = math.ceil(len(train_loader.dataset) / config.batch_size)

    model = ResNet18(config.activation).to(device)
    
    criterion = nn.CrossEntropyLoss()

    if config.optimizer == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    elif config.optimizer == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=config.learning_rate)
    elif config.optimizer == "RMSprop":
        optimizer = optim.RMSprop(model.parameters(), lr=config.learning_rate)
    elif config.optimizer == "Adagrad":
        optimizer = optim.Adagrad(model.parameters(), lr=config.learning_rate)

    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []

    example_ct = 0
    step_ct = 0
    for epoch in range(config.num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        for i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == target).sum().item()
            example_ct += len(data)
            metrics = {
                "train/train_loss": loss.item(),
                "train/train_acc": correct / len(train_loader.dataset),
                "train/epoch": (i+1+(n_steps_per_epoch*epoch)) / (n_steps_per_epoch),
               "train/example_ct": example_ct,
            }
            if i+1 < n_steps_per_epoch:
                wandb.log(metrics)
            
            step_ct += 1
        train_loss.append(running_loss / len(train_loader))
        train_acc.append(correct / len(train_loader.dataset))
        print("Epoch: {}, Train Loss: {}, Train Accuracy: {}".format(epoch+1, train_loss[-1], train_acc[-1]))

        val_loss, val_acc = evaluate(model, test_loader, criterion, test_loss, test_acc, log_images=(epoch == (config.num_epochs - 1)))
        val_metrics = {
            "val/val_loss": val_loss,
            "val/val_acc": val_acc,
            "val/epoch": (i+1+(n_steps_per_epoch*epoch)) / (n_steps_per_epoch),
        }
        wandb.log({**metrics, **val_metrics})
        print("Epoch: {}, Val Loss: {}, Val Accuracy: {}".format(epoch+1, val_loss, val_acc))

    wandb.summary["train_loss"] = train_loss[-1]
    wandb.finish()

Activation: Relu, Optimizer: Adam


Epoch: 1, Train Loss: 0.7560506630308774, Train Accuracy: 0.36564
Epoch: 1, Val Loss: 0.4910200536251068, Val Accuracy: 0.4089
Epoch: 2, Train Loss: 0.4371440184055542, Train Accuracy: 0.42096
Epoch: 2, Val Loss: 0.41333651542663574, Val Accuracy: 0.4264
Epoch: 3, Train Loss: 0.352580013703935, Train Accuracy: 0.43614
Epoch: 3, Val Loss: 0.4195302128791809, Val Accuracy: 0.4284
Epoch: 4, Train Loss: 0.31790465747519414, Train Accuracy: 0.44226
Epoch: 4, Val Loss: 0.36776646971702576, Val Accuracy: 0.434
Epoch: 5, Train Loss: 0.2800515120734974, Train Accuracy: 0.45008
Epoch: 5, Val Loss: 0.36151203513145447, Val Accuracy: 0.4378
Epoch: 6, Train Loss: 0.25473200541217716, Train Accuracy: 0.45404
Epoch: 6, Val Loss: 0.36933648586273193, Val Accuracy: 0.4379
Epoch: 7, Train Loss: 0.2322071880406263, Train Accuracy: 0.45816
Epoch: 7, Val Loss: 0.32162079215049744, Val Accuracy: 0.443
Epoch: 8, Train Loss: 0.21308519626606484, Train Accuracy: 0.46198
Epoch: 8, Val Loss: 0.32724830508232117,

wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.


train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_acc,▂▆▅▅▂▇▇▄▂▂▆▄▄▁▆▆▃██▅▃▂▇▅▄▂▇▆▄▁▁▆▃▃█▅▅▃▇▇
train/train_loss,█▇▇▄▄▃▃▅▃▄▄▃▄▂▄▂▂▂▃▂▁▃▂▂▂▂▃▂▄▃▂▃▁▁▂▂▁▁▂▂
val/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
val/val_acc,▁▄▄▅▅▅▆▆▆▇▇▇▇█▇▇▇▇▇▇█▇█▇█▇██▇█
val/val_loss,█▅▆▄▄▄▂▂▃▁▂▂▂▁▂▁▁▂▂▂▁▂▁▁▁▂▁▁▂▂
train/epoch,29.50128
train/example_ct,750000
train/train_acc,0.48562
train/train_loss,0.05481


Activation: Tanh, Optimizer: RMSprop


...

wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), e

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_acc,▁▃▃▄▂▆▆▄▂▁▆▃▃▁▅▅▃▇▇▅▂▂▇▄▄▂▆▆▄▁▁▆▃▃█▅▅▃█▇
train/train_loss,█▆▅▄▄▄▃▃▃▃▂▃▂▂▃▂▂▃▂▂▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁
val/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
val/val_acc,▁▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████████
val/val_loss,█▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train/epoch,29.50128
train/example_ct,750000
train/train_acc,0.37632
train/train_loss,0.55969


Activation: Sigmoid, Optimizer: Adam


Epoch: 1, Train Loss: 1.6725033868332297, Train Accuracy: 0.3586
Epoch: 1, Val Loss: 1.6193197965621948, Val Accuracy: 0.3874
Epoch: 2, Train Loss: 1.589922431172157, Train Accuracy: 0.40564
Epoch: 2, Val Loss: 1.5945470333099365, Val Accuracy: 0.403
Epoch: 3, Train Loss: 1.573796925496082, Train Accuracy: 0.41534
Epoch: 3, Val Loss: 1.5839979648590088, Val Accuracy: 0.4092
Epoch: 4, Train Loss: 1.565506776984857, Train Accuracy: 0.4227
Epoch: 4, Val Loss: 1.5639013051986694, Val Accuracy: 0.4223
Epoch: 5, Train Loss: 1.5626475476488775, Train Accuracy: 0.42294
Epoch: 5, Val Loss: 1.567138671875, Val Accuracy: 0.4192
Epoch: 6, Train Loss: 1.5568611901633593, Train Accuracy: 0.42924
Epoch: 6, Val Loss: 1.5613850355148315, Val Accuracy: 0.4237
Epoch: 7, Train Loss: 1.5526475346818263, Train Accuracy: 0.43272
Epoch: 7, Val Loss: 1.5727458000183105, Val Accuracy: 0.4208
Epoch: 8, Train Loss: 1.5484499529916413, Train Accuracy: 0.43498
Epoch: 8, Val Loss: 1.561082363128662, Val Accuracy: 0.

train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/example_ct,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/train_acc,▂▆▅▅▂▇▇▄▂▂▆▄▄▁▆▆▃██▅▃▂▇▅▄▂▇▆▄▁▁▆▃▃█▅▅▃▇▇
train/train_loss,█▅▅▃▅▄▃▅▄▄▄▂▃▃▃▃▂▄▂▃▄▂▂▂▃▃▂▂▁▃▂▁▂▂▂▂▂▂▃▃
val/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
val/val_acc,▁▃▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▆▇▇▇█▇▇██▇
val/val_loss,█▆▅▃▄▃▄▃▂▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▂▁▁▁
train/epoch,29.50128
train/example_ct,750000
train/train_acc,0.45424
train/train_loss,1.51952
